# Introduction

The idea behind nornir_tests is to allow for adding tests against the task run and its results.  Many times, a task can run and get some output but that doesn't mean it is successful.  One option is to add additional checks or tasks with some conditional logic to deal with this.  Another is to add tests against the data returned from a task.  That is what nornir_tests is for.

The question is probably why use tests.  Tests were inspired by the same feature in postman.  In postman, a test was something that was done against the data returned from a postman request to determine if the request was actually successful.  This can certainly be added in python using either pure python or additional tasks but there is some elegance and some additional functionality obtained by using tests tied to tasks.  In this intro, I will try to touch on a variety of ideas using tests.  In further tutorials I'll go into more detail on using each of the test types.

## Nornir initialization

This is standard nornir initialization and imports of tasks.  A different library nornir_rich is used for output to provide more detail and highlighting.  Using print_result is also possible if vars is set to include 'tests' attribute.

In [1]:
from nornir_napalm.plugins.tasks import napalm_get, napalm_ping
from nornir_netmiko.tasks import netmiko_send_command
from nornir import InitNornir

nr = InitNornir(
    inventory={
        "plugin": "SimpleInventory",
        "options": {
            "host_file": "data/hosts.yaml",
            "group_file": "data/groups.yaml",
            "defaults_file": "data/defaults.yaml",
        },
    },
)


## Imports of nornir_tests libraries

The print_result in nornir_utils works fine if 'tests' is added to vars.  There is also a modified version of print_result included in nornir_tests that provides a bit better reporting of test records.

In [2]:
from nornir_tests.plugins.tests import regexp, until, timing, jpath
from nornir_tests.plugins.tasks import wrap_task
from nornir_tests.plugins.functions import print_result

## Run first wrapped task

In the first example, a vyos router interface facts are obtained using napalm and a test is run to validate the result.

In [3]:
vyos = nr.filter(name='vyos')

result = vyos.run(
    task=wrap_task(napalm_get),
    getters=['interfaces'],
    tests=[
        jpath(path='interfaces.eth0.is_up', assertion='is_true'),
    ]
)

print_result(result, vars=['result', 'tests'])

napalm_get**********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{'interfaces':{'eth0':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'08:00:27:e0:28:63',
'mtu':-1,
'speed':0},
'lo':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'00:00:00:00:00:00',
'mtu':-1,
'speed':0}}}
P JpathRecord - {'assertion': 'is_true',
 'path': 'interfaces.eth0.is_up',
 'result_attr': 'result'}
{'matches': ['interfaces.eth0.is_up']}
^^^^ END napalm_get ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Run ping with timing requirements

Next a ping is sent to the vyos device using the timing test.  As can be seen in the output, this gives runtime of the task and can be enhanced with min_run_time and max_run_time to affect the task result if the fail_task argument is used.  If a failure occurs in the test and fail_task is set to True, then the task will be marked as failed.

In [4]:
result = vyos.run(
    task=wrap_task(napalm_ping),
    dest='192.168.99.1',
    tests=[
        timing(),
    ]
)
print_result(result, vars=['result', 'tests'])

napalm_ping*********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_ping ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{'success':{'packet_loss':0,
'probes_sent':5,
'results':[{'ip_address': '192.168.99.1', 'rtt': 4.288}],
'rtt_avg':4.288,
'rtt_max':6.675,
'rtt_min':3.434,
'rtt_stddev':1.216}}
P TimingRecord - {'max_run_time': 9223372036854775807}
{'run_time': 4.914522171020508,
 't0': 1600801169.7141054,
 't1': 1600801174.6286275}
^^^^ END napalm_ping ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Get unfailed task to fail due to assertion

The next example shows how the result failed state can be affected by a test.

In [5]:
result = vyos.run(
    task=wrap_task(napalm_ping),
    dest='192.168.99.1',
    tests=[
        timing(max_run_time=1, fail_task=True),
    ]
)
print_result(result, vars=['result', 'tests'])

napalm_ping*********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_ping ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv ERROR
{'success':{'packet_loss':0,
'probes_sent':5,
'results':[{'ip_address': '192.168.99.1', 'rtt': 3.342}],
'rtt_avg':3.342,
'rtt_max':4.664,
'rtt_min':2.631,
'rtt_stddev':0.696}}
F TimingRecord - {'fail_task': True, 'max_run_time': 1}
{'run_time': 4.914187431335449,
 't0': 1600801174.8475542,
 't1': 1600801179.7617416}
^^^^ END napalm_ping ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


## Combining tests

Tests can be stacked and the entire task and associated tests can be retried using until.  The next test shows how a rebooted device can be retried until it comes back up using only tests.

In [6]:
nr.data.reset_failed_hosts()

print_result(
    vyos.run(
        task=netmiko_send_command,
        command_string='reboot now'
    )
)

result = vyos.run(
    task=wrap_task(napalm_get),
    getters=['interfaces'],
    tests=[
        jpath(path='interfaces.eth0.is_up', assertion='is_true', fail_task=True),
        until(initial_delay=15, retries=10, delay=15, reset_conns=True),
    ]
)
print_result(result, vars=['result', 'tests'])

netmiko_send_command************************************************************
* vyos ** changed : False ******************************************************
vvvv netmiko_send_command ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
^^^^ END netmiko_send_command ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
napalm_get**********************************************************************
* vyos ** changed : False ******************************************************
vvvv napalm_get ** changed : False vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv INFO
{'interfaces':{'eth0':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'08:00:27:e0:28:63',
'mtu':-1,
'speed':0},
'lo':{'description':'',
'is_enabled':True,
'is_up':True,
'last_flapped':-1.0,
'mac_address':'00:00:00:00:00:00',
'mtu':-1,
'speed':0}}}
P JpathRecord - {'assertion': 'is_true',
 'fail_task': True,
 'path': 'interfaces.eth0.is_up',
 'result_attr': 'result'}
{'matches': ['

The result above shows that it must have failed on its first try but around the one minute mark it was able to pass the jpath test and so until finished up.

Check out all the other tutorials to see each in more detail.